In [1]:
import cv2
import numpy as np
import os
import urllib.request
from collections import deque

# --- CONFIGURATION ---
FARNEBACK_PARAMS = dict(
    pyr_scale=0.5,
    levels=3,
    winsize=15,
    iterations=3,
    poly_n=5,
    poly_sigma=1.2,
    flags=0
)

# Paramètres d'analyse
MAGNITUDE_THRESHOLD = 2.0  # Seuil de bruit pour le mouvement
HISTORY_LENGTH = 30        # Nombre de frames pour l'analyse temporelle (approx 1 sec)
WAVE_THRESHOLD = 3         # Nombre d'inversions de direction pour valider un "Coucou"

# Paramètres YOLO
YOLO_CONFIG = "yolov3-tiny.cfg"
YOLO_WEIGHTS = "yolov3-tiny.weights"
YOLO_CLASSES = "coco.names"
CONFIDENCE_THRESHOLD = 0.3
NMS_THRESHOLD = 0.4

# --- CLASSE D'ANALYSE DE GESTES ---
class GestureAnalyzer:
    def __init__(self, maxlen=30):
        self.history_fx = deque(maxlen=maxlen) # Vitesse horizontale
        self.history_speed = deque(maxlen=maxlen) # Vitesse globale
        
    def update(self, avg_fx, avg_speed):
        self.history_fx.append(avg_fx)
        self.history_speed.append(avg_speed)
        
    def detect_gesture(self):
        if len(self.history_speed) < 10:
            return "Analyse..."

        # 1. Analyse de l'immobilité (STOP)
        recent_speeds = list(self.history_speed)[-10:] # Dernières 10 frames
        if np.mean(recent_speeds) < 1.0:
            return "STOP (Immobile)"
            
        # 2. Analyse de l'oscillation (COUCOU / WAVE)
        # On regarde si la vitesse horizontale (fx) change de signe souvent
        fx_array = np.array(self.history_fx)
        
        # On ne considère que les mouvements significatifs pour éviter le bruit autour de 0
        significant_moves = fx_array[np.abs(fx_array) > 0.5]
        
        if len(significant_moves) > 5:
            # Compte les changements de signe (passages par zéro)
            zero_crossings = np.sum(np.abs(np.diff(np.sign(significant_moves)))) / 2
            
            if zero_crossings >= WAVE_THRESHOLD:
                return "COUCOU (Wave) 👋"
        
        return "Mouvement..."

# --- GESTION YOLO ---
def download_yolo_files():
    """Télécharge les fichiers YOLOv3-tiny si absents."""
    base_url = "https://pjreddie.com/media/files/"
    if not os.path.exists(YOLO_WEIGHTS):
        print(f"Téléchargement de {YOLO_WEIGHTS}...")
        try:
            urllib.request.urlretrieve(base_url + YOLO_WEIGHTS, YOLO_WEIGHTS)
        except Exception as e:
            print(f"Erreur téléchargement poids: {e}")
            return False

    # Note: Le .cfg et coco.names sont souvent requis. 
    # Pour simplifier, si on ne les a pas, on retournera False dans load_yolo
    # Dans un vrai projet, il faudrait aussi les télécharger depuis un repo github raw.
    return True

def load_yolo():
    # Vérification basique des fichiers (ici on assume que le user a les fichiers ou qu'on a pu télécharger les poids)
    # Pour ce script autonome, on va essayer de charger, sinon on retourne None
    if not os.path.exists(YOLO_WEIGHTS):
        print("Fichiers YOLO manquants. Passage en mode MANUEL.")
        return None, None, None
        
    # Création d'une config minimale si absente (astuce pour rendre le script portable)
    if not os.path.exists(YOLO_CONFIG):
        # On ne peut pas inventer la config, il faut que l'utilisateur l'ait.
        # Fallback manuel si config manquante.
        print(f"Fichier {YOLO_CONFIG} manquant. Passage en mode MANUEL.")
        return None, None, None

    try:
        net = cv2.dnn.readNetFromDarknet(YOLO_CONFIG, YOLO_WEIGHTS)
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
        
        layer_names = net.getLayerNames()
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
        return net, output_layers, [] # On ignore les classes names pour simplifier
    except Exception as e:
        print(f"Erreur chargement YOLO: {e}")
        return None, None, None

def detect_objects_yolo(frame, net, output_layers):
    height, width = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            # On détecte "Personne" (ID 0)
            if confidence > CONFIDENCE_THRESHOLD and class_id == 0:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
    
    if len(indices) > 0:
        # Retourne la boîte la plus confiante
        i = indices[0]
        # Si i est une liste/tuple (dépend version OpenCV), on extrait l'index
        if isinstance(i, (tuple, list, np.ndarray)):
            i = i.item() 
        return boxes[i]
    return None

# --- ANALYSE DU FLOT (Inchangé mais retourne plus d'infos) ---
def analyze_flow_in_roi(flow, roi_gray):
    fx, fy = flow[..., 0], flow[..., 1]
    mag, ang = cv2.cartToPolar(fx, fy, angleInDegrees=True)
    motion_mask = mag > MAGNITUDE_THRESHOLD
    
    if np.count_nonzero(motion_mask) < 10:
        return 0.0, 0.0, 0.0, motion_mask, mag # Ajout fx moyen

    avg_speed = np.mean(mag[motion_mask])
    avg_fx = np.mean(fx[motion_mask]) # Vitesse horizontale moyenne signée
    avg_fy = np.mean(fy[motion_mask])
    _, avg_angle = cv2.cartToPolar(np.array([avg_fx]), np.array([avg_fy]), angleInDegrees=True)
    
    return avg_speed, avg_fx, avg_angle[0], motion_mask, mag

def get_cardinal_direction(angle):
    directions = ["Est", "Sud-Est", "Sud", "Sud-Ouest", "Ouest", "Nord-Ouest", "Nord", "Nord-Est"]
    idx = int((angle + 22.5) / 45.0) % 8
    return directions[idx]

# --- VARIABLES GLOBALES SOURIS ---
selection_rect = None
drawing = False
ix, iy = -1, -1

def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, selection_rect
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
        selection_rect = None
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            selection_rect = (min(ix, x), min(iy, y), abs(x - ix), abs(y - iy))
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        selection_rect = (min(ix, x), min(iy, y), abs(x - ix), abs(y - iy))

# --- MAIN LOOP ---
def main():
    global selection_rect
    
    # 1. Tentative de chargement YOLO
    # On essaye de télécharger les poids si absents (pour démo)
    # download_yolo_files() # Décommenter pour tenter le téléchargement auto
    
    net, output_layers, _ = load_yolo()
    use_yolo = (net is not None)
    
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Erreur webcam")
        return

    window_name = 'Analyse Mouvement + Gestes'
    cv2.namedWindow(window_name)
    cv2.setMouseCallback(window_name, draw_rectangle)

    prev_roi_frame = None
    gesture_analyzer = GestureAnalyzer(maxlen=HISTORY_LENGTH)
    
    mode_text = "MODE: YOLO (Auto)" if use_yolo else "MODE: MANUEL (Souris)"
    print(f"Démarrage. {mode_text}")
    if not use_yolo:
        print("Note: Pour utiliser YOLO, placez 'yolov3-tiny.weights' et 'yolov3-tiny.cfg' dans le dossier.")

    while True:
        ret, frame = cap.read()
        if not ret: break
        
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        display_frame = frame.copy()
        
        current_box = None

        # --- ÉTAPE 1: DÉTECTION (YOLO ou MANUEL) ---
        if use_yolo:
            # On détecte toutes les quelques frames pour la perf (ici à chaque frame pour fluidité démo)
            detected_box = detect_objects_yolo(frame, net, output_layers)
            if detected_box:
                selection_rect = tuple(detected_box) # Mise à jour auto
                current_box = selection_rect
        else:
            # Mode manuel
            if selection_rect is not None and selection_rect[2] > 0:
                current_box = selection_rect

        # --- ÉTAPE 2: TRAITEMENT ROI ---
        if current_box:
            x, y, w, h = current_box
            # Vérification des bornes
            x, y = max(0, x), max(0, y)
            w = min(w, frame.shape[1] - x)
            h = min(h, frame.shape[0] - y)
            
            if w > 10 and h > 10:
                cv2.rectangle(display_frame, (x, y), (x + w, y + h), (255, 100, 0), 2)
                roi_gray = gray[y:y+h, x:x+w]
                
                if prev_roi_frame is not None and prev_roi_frame.shape == roi_gray.shape:
                    # Calcul Flot
                    flow = cv2.calcOpticalFlowFarneback(prev_roi_frame, roi_gray, None, **FARNEBACK_PARAMS)
                    
                    # Analyse Instantanée
                    speed, avg_fx, angle, mask, _ = analyze_flow_in_roi(flow, roi_gray)
                    
                    # Mise à jour Analyseur Gestes
                    gesture_analyzer.update(avg_fx, speed)
                    gesture_detected = gesture_analyzer.detect_gesture()
                    
                    # Visualisation Vecteurs (Sous-échantillonnage)
                    step = 10
                    for r in range(0, h, step):
                        for c in range(0, w, step):
                            if mask[r, c]:
                                cv2.arrowedLine(display_frame, 
                                              (x+c, y+r), 
                                              (int(x+c+flow[r,c,0]), int(y+r+flow[r,c,1])), 
                                              (0, 255, 0), 1, tipLength=0.3)
                    
                    # Affichage Infos
                    cv2.putText(display_frame, f"Geste: {gesture_detected}", (x, y-30), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
                    
                    direction = get_cardinal_direction(angle) if speed > 0.5 else "-"
                    cv2.putText(display_frame, f"Vit: {speed:.1f} | Dir: {direction}", (x, y-10), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)

                prev_roi_frame = roi_gray.copy()
            else:
                prev_roi_frame = None
        else:
            prev_roi_frame = None
            if not use_yolo:
                cv2.putText(display_frame, "Dessinez un cadre sur l'objet", (20, 50), 
                          cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                cv2.putText(display_frame, "Recherche de personne...", (20, 50), 
                          cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # UI Info
        cv2.putText(display_frame, mode_text, (10, frame.shape[0] - 10), 
                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

        if drawing and selection_rect:
             dx, dy, dw, dh = selection_rect
             cv2.rectangle(display_frame, (dx, dy), (dx + dw, dy + dh), (0, 255, 255), 1)

        cv2.imshow(window_name, display_frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'): break
        if key == ord('r'): selection_rect = None # Reset manuel
        if key == ord('m'): # Bascule forcée manuel/auto
            use_yolo = not use_yolo
            selection_rect = None
            mode_text = "MODE: YOLO (Auto)" if use_yolo else "MODE: MANUEL"

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Fichiers YOLO manquants. Passage en mode MANUEL.
Démarrage. MODE: MANUEL (Souris)
Note: Pour utiliser YOLO, placez 'yolov3-tiny.weights' et 'yolov3-tiny.cfg' dans le dossier.


C:\Users\moura\AppData\Local\Temp\ipykernel_45744\2516172946.py:166: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  idx = int((angle + 22.5) / 45.0) % 8
